# Code Smell Detection

    In this Notebook we're going to use https://zenodo.org/record/3590102#.YvEOaHZByUk datasets to train a ML algorithm for smell prediction.

    In the first steps we import our Pandas module to handle the datasets and its modification.

In [3]:
import pandas as pd

# Read the csv file
df = pd.read_csv('D:\\Articles\\Code Smells\\CodeSmells\\Smell Project\\Datasets\\MainDataset.csv')

# Print it out if you want
print(df.shape, "\n")
print("This dataset consists of following features:\n", list(df.columns), sep="") # getting the features' name
df

(14853, 15) 

This dataset consists of following features:
['id', 'reviewer_id', 'sample_id', 'smell', 'severity', 'review_timestamp', 'type', 'code_name', 'repository', 'commit_hash', 'path', 'start_line', 'end_line', 'link', 'is_from_industry_relevant_project']


,id,reviewer_id,sample_id,smell,severity,review_timestamp,type,code_name,repository,commit_hash,path,start_line,end_line,link,is_from_industry_relevant_project
0,526,6,5771277,feature envy,none,34:53.0,function,org.apache.syncope.client.ui.commons.ConnIdSpe...,git@github.com:apache/syncope.git,114c412afbfba24ffb4fbc804e5308a823a16a78,/client/idrepo/ui/src/main/java/org/apache/syn...,35,37,https://github.com/apache/syncope/blob/114c412...,1.0
1,527,6,5771277,long method,none,34:53.0,function,org.apache.syncope.client.ui.commons.ConnIdSpe...,git@github.com:apache/syncope.git,114c412afbfba24ffb4fbc804e5308a823a16a78,/client/idrepo/ui/src/main/java/org/apache/syn...,35,37,https://github.com/apache/syncope/blob/114c412...,1.0
2,528,6,5786929,blob,critical,37:38.1,class,org.apache.tez.runtime.library.common.writers....,git@github.com:apache/tez.git,d5675c332497c1ac1dedefdf91e87476b5c0d7a9,/tez-runtime-library/src/main/java/org/apache/...,89,1427,https://github.com/apache/tez/blob/d5675c33249...,1.0
3,529,6,5786929,data class,critical,37:38.1,class,org.apache.tez.runtime.library.common.writers....,git@github.com:apache/tez.git,d5675c332497c1ac1dedefdf91e87476b5c0d7a9,/tez-runtime-library/src/main/java/org/apache/...,89,1427,https://github.com/apache/tez/blob/d5675c33249...,1.0
4,530,6,5788107,feature envy,none,37:49.6,function,org.apache.tika.parser.ocr.TesseractOCRConfig#...,git@github.com:apache/tika.git,4131c6e30f2e0eb1feb85e0f7576531d4e830468,/tika-parsers/src/main/java/org/apache/tika/pa...,531,534,https://github.com/apache/tika/blob/4131c6e30f...,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14848,15348,20,8968865,blob,none,24:46.5,class,org.apache.hive.hplsql.Signal,git@github.com:apache/hive.git,2fa22bf360898dc8fd1408bfcc96e1c6aeaf9a53,/hplsql/src/main/java/org/apache/hive/hplsql/S...,24,48,https://github.com/apache/hive/blob/2fa22bf360...,1.0
14849,15349,20,9474095,data class,none,24:54.3,class,org.apache.hadoop.yarn.service.provider.Abstra...,git@github.com:apache/hadoop.git,128dd91e10080bdcbcd7d555fa3c4105e55a6b51,/hadoop-yarn-project/hadoop-yarn/hadoop-yarn-a...,38,150,https://github.com/apache/hadoop/blob/128dd91e...,1.0
14850,15350,20,9474095,blob,none,24:54.3,class,org.apache.hadoop.yarn.service.provider.Abstra...,git@github.com:apache/hadoop.git,128dd91e10080bdcbcd7d555fa3c4105e55a6b51,/hadoop-yarn-project/hadoop-yarn/hadoop-yarn-a...,38,150,https://github.com/apache/hadoop/blob/128dd91e...,1.0
14851,15351,20,6293245,data class,none,25:13.0,class,org.eclipse.jetty.plus.security.DataSourceLogi...,git@github.com:eclipse/jetty.project.git,65528f76c5ef6ddca11385f9721c8f0bc5f2eed7,/jetty-plus/src/main/java/org/eclipse/jetty/pl...,52,530,https://github.com/eclipse/jetty.project/blob/...,1.0


    There are too many none value for severity feature. Value 'none' for severity means that none of the code reviewers know whether the code has smell or not so in the next cell we're going to delete the rows containing none value for severity as they are useless in our computations.

In [4]:
dfWithoutNone = df[df['severity'] != "none"]
dfWithoutNone.to_csv("D:\\Articles\\Code Smells\\CodeSmells\\Smell Project\\Datasets\\DatasetWithoutNone.csv")
dfWithoutNone.shape

(3301, 15)

In [5]:
dfWithoutNone

,id,reviewer_id,sample_id,smell,severity,review_timestamp,type,code_name,repository,commit_hash,path,start_line,end_line,link,is_from_industry_relevant_project
2,528,6,5786929,blob,critical,37:38.1,class,org.apache.tez.runtime.library.common.writers....,git@github.com:apache/tez.git,d5675c332497c1ac1dedefdf91e87476b5c0d7a9,/tez-runtime-library/src/main/java/org/apache/...,89,1427,https://github.com/apache/tez/blob/d5675c33249...,1.0
3,529,6,5786929,data class,critical,37:38.1,class,org.apache.tez.runtime.library.common.writers....,git@github.com:apache/tez.git,d5675c332497c1ac1dedefdf91e87476b5c0d7a9,/tez-runtime-library/src/main/java/org/apache/...,89,1427,https://github.com/apache/tez/blob/d5675c33249...,1.0
25,551,6,5822090,blob,minor,42:50.3,class,org.apache.uima.ruta.action.TrimAction,git@github.com:apache/uima-ruta.git,08efb111a775b4ba01cf048ac699aa9c22188325,/ruta-core/src/main/java/org/apache/uima/ruta/...,37,136,https://github.com/apache/uima-ruta/blob/08efb...,1.0
28,554,6,5828468,blob,major,44:23.9,class,org.apache.unomi.mailchimp.services.internal.M...,git@github.com:apache/unomi.git,1745c5221401a05e66ebf80fbe9cce2790442b3a,/extensions/unomi-mailchimp/services/src/main/...,40,434,https://github.com/apache/unomi/blob/1745c5221...,1.0
29,555,6,5828468,data class,minor,44:23.9,class,org.apache.unomi.mailchimp.services.internal.M...,git@github.com:apache/unomi.git,1745c5221401a05e66ebf80fbe9cce2790442b3a,/extensions/unomi-mailchimp/services/src/main/...,40,434,https://github.com/apache/unomi/blob/1745c5221...,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14835,15335,20,7916535,long method,major,22:27.6,function,org.apache.xalan.xsltc.runtime.ObjectFactory.f...,git@github.com:apache/servicemix-bundles.git,5f2c7727f71c167997947ad4604f9b8200952af2,/xalan-2.7.2/src/main/java/org/apache/xalan/xs...,477,529,https://github.com/apache/servicemix-bundles/b...,0.0
14838,15338,20,8169113,data class,major,22:46.3,class,org.apache.activemq.filter.BinaryExpression,git@github.com:apache/activemq.git,ccf56875b0660214e0a61bd2f8adc418143551fc,/activemq-client/src/main/java/org/apache/acti...,26,96,https://github.com/apache/activemq/blob/ccf568...,1.0
14843,15343,20,7638207,blob,minor,24:31.2,class,org.springframework.ide.eclipse.boot.launch.cl...,git@github.com:spring-projects/spring-ide.git,915fe9bffd50db45ee0b8fb993416e45dee68179,/plugins/org.springframework.ide.eclipse.boot....,54,297,https://github.com/spring-projects/spring-ide/...,1.0
14847,15347,20,8968865,data class,major,24:46.5,class,org.apache.hive.hplsql.Signal,git@github.com:apache/hive.git,2fa22bf360898dc8fd1408bfcc96e1c6aeaf9a53,/hplsql/src/main/java/org/apache/hive/hplsql/S...,24,48,https://github.com/apache/hive/blob/2fa22bf360...,1.0


    Let's seprate and categorize the type of smells existing in this dataset. With the following code we can get the varies of the smell in this dataset and it is 4 different smell as we expected.

In [6]:
dfWithoutNone["smell"].unique()

array(['blob', 'data class', 'long method', 'feature envy'], dtype=object)

    In the following we are going to seprate these smells and data and adding them to their specified datasets.

In [7]:
dfFeatureEnvy = dfWithoutNone[dfWithoutNone["smell"] == 'feature envy']
dfFeatureEnvy.to_csv("D:\\Articles\\Code Smells\\CodeSmells\\Smell Project\\Datasets\\DatasetWithoutNoneFeatureEnvy.csv")
dfFeatureEnvy

,id,reviewer_id,sample_id,smell,severity,review_timestamp,type,code_name,repository,commit_hash,path,start_line,end_line,link,is_from_industry_relevant_project
44,570,7,5839980,feature envy,minor,46:31.2,function,org.apache.tomcat.util.descriptor.web.ContextR...,git@github.com:apache/tomcat.git,a9c1a0661198d9ba37c1facd8385fe05d538c4ad,/java/org/apache/tomcat/util/descriptor/web/Co...,94,121,https://github.com/apache/tomcat/blob/a9c1a066...,1.0
47,573,6,5855589,feature envy,minor,47:09.4,function,org.apache.catalina.ha.session.DeltaSession#en...,git@github.com:apache/tomcat.git,a9c1a0661198d9ba37c1facd8385fe05d538c4ad,/java/org/apache/catalina/ha/session/DeltaSess...,412,418,https://github.com/apache/tomcat/blob/a9c1a066...,1.0
85,613,6,5922177,feature envy,minor,51:45.2,function,org.apache.zeppelin.realm.jwt.KnoxJwtRealm#val...,git@github.com:apache/zeppelin.git,4219d552349f8f7f3e6de34505b8a8ae9835f98b,/zeppelin-server/src/main/java/org/apache/zepp...,130,156,https://github.com/apache/zeppelin/blob/4219d5...,1.0
94,622,6,5939954,feature envy,major,55:26.3,function,org.apache.zookeeper.graph.JsonGenerator.JsonG...,git@github.com:apache/zookeeper.git,07c3aaf3d723fb3144c0aedc0c2b655325df70e9,/zookeeper-contrib/zookeeper-contrib-loggraph/...,75,213,https://github.com/apache/zookeeper/blob/07c3a...,1.0
103,631,6,6133230,feature envy,minor,56:52.6,function,org.eclipse.dltk.internal.ui.refactoring.reorg...,git@github.com:eclipse/dltk.core.git,ff5ee79db0c2234347cb3f956de9b31047f43613,/core/plugins/org.eclipse.dltk.ui/ui refactori...,39,46,https://github.com/eclipse/dltk.core/blob/ff5e...,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14770,15270,21,6863919,feature envy,minor,29:24.1,function,com.google.errorprone.bugpatterns.inject.Injec...,git@github.com:google/error-prone.git,61cb540c08ec63faa56dccce00049cff1f8b41ea,/core/src/main/java/com/google/errorprone/bugp...,72,128,https://github.com/google/error-prone/blob/61c...,1.0
14782,15282,21,8861277,feature envy,minor,35:28.3,function,org.apache.airavata.registry.core.app.catalog....,git@github.com:apache/airavata.git,391843a00eefa7b6213e845f2f044b4e042894d5,/modules/registry/registry-core/src/main/java/...,522,556,https://github.com/apache/airavata/blob/391843...,1.0
14811,15311,21,4081252,feature envy,critical,40:45.1,function,org.apache.hyracks.storage.am.lsm.rtree.datafl...,git@github.com:apache/asterixdb.git,223d13a06c4a4a58408aeac19674ac1f36f5ff35,/hyracks-fullstack/hyracks/hyracks-storage-am-...,96,109,https://github.com/apache/asterixdb/blob/223d1...,1.0
14818,15318,21,8389037,feature envy,minor,54:22.8,function,org.apache.camel.component.disruptor.Disruptor...,git@github.com:apache/camel.git,8a85a70643c4d6eec2d3abddeea44ecb06c2f486,/components/camel-disruptor/src/main/java/org/...,64,108,https://github.com/apache/camel/blob/8a85a7064...,1.0


In [8]:
dfBlob = dfWithoutNone[dfWithoutNone["smell"] == 'blob']
dfBlob.to_csv("D:\\Articles\\Code Smells\\CodeSmells\\Smell Project\\Datasets\\DatasetWithoutNoneBlob.csv")
dfBlob

,id,reviewer_id,sample_id,smell,severity,review_timestamp,type,code_name,repository,commit_hash,path,start_line,end_line,link,is_from_industry_relevant_project
2,528,6,5786929,blob,critical,37:38.1,class,org.apache.tez.runtime.library.common.writers....,git@github.com:apache/tez.git,d5675c332497c1ac1dedefdf91e87476b5c0d7a9,/tez-runtime-library/src/main/java/org/apache/...,89,1427,https://github.com/apache/tez/blob/d5675c33249...,1.0
25,551,6,5822090,blob,minor,42:50.3,class,org.apache.uima.ruta.action.TrimAction,git@github.com:apache/uima-ruta.git,08efb111a775b4ba01cf048ac699aa9c22188325,/ruta-core/src/main/java/org/apache/uima/ruta/...,37,136,https://github.com/apache/uima-ruta/blob/08efb...,1.0
28,554,6,5828468,blob,major,44:23.9,class,org.apache.unomi.mailchimp.services.internal.M...,git@github.com:apache/unomi.git,1745c5221401a05e66ebf80fbe9cce2790442b3a,/extensions/unomi-mailchimp/services/src/main/...,40,434,https://github.com/apache/unomi/blob/1745c5221...,1.0
64,592,6,5884892,blob,major,50:05.9,class,org.apache.usergrid.batch.service.JobScheduler...,git@github.com:apache/usergrid.git,ac1e6e4035f9307b871478ed47246cf92cfd5f7f,/stack/core/src/main/java/org/apache/usergrid/...,55,450,https://github.com/apache/usergrid/blob/ac1e6e...,1.0
86,614,6,5938889,blob,minor,53:01.7,class,org.apache.zookeeper.inspector.gui.ZooInspecto...,git@github.com:apache/zookeeper.git,07c3aaf3d723fb3144c0aedc0c2b655325df70e9,/zookeeper-contrib/zookeeper-contrib-zooinspec...,46,258,https://github.com/apache/zookeeper/blob/07c3a...,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14827,15327,20,5247450,blob,minor,17:04.7,class,org.apache.oozie.tools.diag.AppInfoCollector,git@github.com:apache/oozie.git,491e73ee9f941dfb25dfe92121fb033295ef42ee,/tools/src/main/java/org/apache/oozie/tools/di...,45,443,https://github.com/apache/oozie/blob/491e73ee9...,1.0
14829,15329,20,6454251,blob,minor,17:22.0,class,org.tukaani.xz.SeekableXZInputStream,git@github.com:eclipse/packagedrone.git,3869c1643cdc6f7cb8b26097a7b6994683b13d7e,/bundles/org.tukani.xz/src/org/tukaani/xz/Seek...,76,1152,https://github.com/eclipse/packagedrone/blob/3...,1.0
14834,15334,20,8528735,blob,major,22:18.9,class,com.cloud.network.ExternalLoadBalancerDeviceMa...,git@github.com:apache/cloudstack.git,8d3feb100aab4a45b31a789f444038b892161eec,/server/src/main/java/com/cloud/network/Extern...,141,1311,https://github.com/apache/cloudstack/blob/8d3f...,1.0
14843,15343,20,7638207,blob,minor,24:31.2,class,org.springframework.ide.eclipse.boot.launch.cl...,git@github.com:spring-projects/spring-ide.git,915fe9bffd50db45ee0b8fb993416e45dee68179,/plugins/org.springframework.ide.eclipse.boot....,54,297,https://github.com/spring-projects/spring-ide/...,1.0


In [9]:
dfDataClass = dfWithoutNone[dfWithoutNone["smell"] == 'data class']
dfDataClass.to_csv("D:\\Articles\\Code Smells\\CodeSmells\\Smell Project\\Datasets\\DatasetWithoutNoneDataClass.csv")
dfDataClass

,id,reviewer_id,sample_id,smell,severity,review_timestamp,type,code_name,repository,commit_hash,path,start_line,end_line,link,is_from_industry_relevant_project
3,529,6,5786929,data class,critical,37:38.1,class,org.apache.tez.runtime.library.common.writers....,git@github.com:apache/tez.git,d5675c332497c1ac1dedefdf91e87476b5c0d7a9,/tez-runtime-library/src/main/java/org/apache/...,89,1427,https://github.com/apache/tez/blob/d5675c33249...,1.0
29,555,6,5828468,data class,minor,44:23.9,class,org.apache.unomi.mailchimp.services.internal.M...,git@github.com:apache/unomi.git,1745c5221401a05e66ebf80fbe9cce2790442b3a,/extensions/unomi-mailchimp/services/src/main/...,40,434,https://github.com/apache/unomi/blob/1745c5221...,1.0
34,560,7,5827650,data class,minor,44:58.5,class,org.apache.uima.ruta.cde.ui.DocumentTableLabel...,git@github.com:apache/uima-ruta.git,08efb111a775b4ba01cf048ac699aa9c22188325,/ruta-ep-addons/src/main/java/org/apache/uima/...,29,85,https://github.com/apache/uima-ruta/blob/08efb...,1.0
91,619,7,5935402,data class,major,53:35.8,class,org.apache.zeppelin.spark.SparkVersion,git@github.com:apache/zeppelin.git,4219d552349f8f7f3e6de34505b8a8ae9835f98b,/spark/interpreter/src/main/java/org/apache/ze...,25,123,https://github.com/apache/zeppelin/blob/4219d5...,1.0
99,627,6,6002683,data class,major,55:55.7,class,org.apache.wss4j.policy.model.Trust13,git@github.com:apache/wss4j.git,5c2cbd7e7377296edab02ab79fc98b84fb47f825,/policy/src/main/java/org/apache/wss4j/policy/...,30,170,https://github.com/apache/wss4j/blob/5c2cbd7e7...,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14768,15268,21,6293245,data class,major,56:47.9,class,org.eclipse.jetty.plus.security.DataSourceLogi...,git@github.com:eclipse/jetty.project.git,65528f76c5ef6ddca11385f9721c8f0bc5f2eed7,/jetty-plus/src/main/java/org/eclipse/jetty/pl...,52,530,https://github.com/eclipse/jetty.project/blob/...,1.0
14784,15284,21,6483071,data class,major,35:43.9,class,com.ibm.jpp.om.Builder,git@github.com:eclipse/openj9.git,4911084853eb75b20e037c434ad4521b7317ebfb,/sourcetools/com.ibm.jpp.preprocessor/com/ibm/...,41,806,https://github.com/eclipse/openj9/blob/4911084...,1.0
14800,15300,21,4503985,data class,minor,38:39.1,class,org.apache.fluo.command.FluoScan.ScanOptions,git@github.com:apache/fluo.git,8e06204d4167651e2d3b5219b8c1397644e6ba6e,/modules/command/src/main/java/org/apache/fluo...,33,122,https://github.com/apache/fluo/blob/8e06204d41...,1.0
14838,15338,20,8169113,data class,major,22:46.3,class,org.apache.activemq.filter.BinaryExpression,git@github.com:apache/activemq.git,ccf56875b0660214e0a61bd2f8adc418143551fc,/activemq-client/src/main/java/org/apache/acti...,26,96,https://github.com/apache/activemq/blob/ccf568...,1.0


In [10]:
dfLongMethod = dfWithoutNone[dfWithoutNone["smell"] == 'long method']
dfLongMethod.to_csv("D:\\Articles\\Code Smells\\CodeSmells\\Smell Project\\Datasets\\DatasetWithoutNoneLongMethod.csv")
dfLongMethod

,id,reviewer_id,sample_id,smell,severity,review_timestamp,type,code_name,repository,commit_hash,path,start_line,end_line,link,is_from_industry_relevant_project
39,565,6,5840527,long method,minor,45:41.3,function,org.apache.el.parser.ELParser#DynamicExpression,git@github.com:apache/tomcat.git,a9c1a0661198d9ba37c1facd8385fe05d538c4ad,/java/org/apache/el/parser/ELParser.java,140,168,https://github.com/apache/tomcat/blob/a9c1a066...,1.0
84,612,6,5922177,long method,minor,51:45.2,function,org.apache.zeppelin.realm.jwt.KnoxJwtRealm#val...,git@github.com:apache/zeppelin.git,4219d552349f8f7f3e6de34505b8a8ae9835f98b,/zeppelin-server/src/main/java/org/apache/zepp...,130,156,https://github.com/apache/zeppelin/blob/4219d5...,1.0
95,623,6,5939954,long method,critical,55:26.3,function,org.apache.zookeeper.graph.JsonGenerator.JsonG...,git@github.com:apache/zookeeper.git,07c3aaf3d723fb3144c0aedc0c2b655325df70e9,/zookeeper-contrib/zookeeper-contrib-loggraph/...,75,213,https://github.com/apache/zookeeper/blob/07c3a...,1.0
152,682,7,6182357,long method,minor,08:50.9,function,org.eclipse.che.infrastructure.docker.client.j...,git@github.com:eclipse/che.git,c5498c2ac562cd8a2fc79a6bb0446d291f05a201,/infrastructures/docker/docker-client/src/main...,90,116,https://github.com/eclipse/che/blob/c5498c2ac5...,1.0
180,710,7,6236496,long method,major,17:30.9,function,org.eclipse.elk.core.meta.ide.contentassist.an...,git@github.com:eclipse/elk.git,9a87764f00d863463b1be6de1920d8aa3c3ade70,/plugins/org.eclipse.elk.core.meta.ui/src-gen/...,45156,45191,https://github.com/eclipse/elk/blob/9a87764f00...,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14769,15269,21,6863919,long method,minor,29:24.1,function,com.google.errorprone.bugpatterns.inject.Injec...,git@github.com:google/error-prone.git,61cb540c08ec63faa56dccce00049cff1f8b41ea,/core/src/main/java/com/google/errorprone/bugp...,72,128,https://github.com/google/error-prone/blob/61c...,1.0
14817,15317,21,8389037,long method,minor,54:22.8,function,org.apache.camel.component.disruptor.Disruptor...,git@github.com:apache/camel.git,8a85a70643c4d6eec2d3abddeea44ecb06c2f486,/components/camel-disruptor/src/main/java/org/...,64,108,https://github.com/apache/camel/blob/8a85a7064...,1.0
14819,15319,21,3942097,long method,minor,54:32.0,function,org.apache.aries.spifly.statictool.Main.weaveD...,git@github.com:apache/aries.git,52293d20268de7c98833846ded2b70d6476773de,/spi-fly/spi-fly-static-tool/src/main/java/org...,173,223,https://github.com/apache/aries/blob/52293d202...,1.0
14826,15326,20,8861277,long method,minor,16:50.9,function,org.apache.airavata.registry.core.app.catalog....,git@github.com:apache/airavata.git,391843a00eefa7b6213e845f2f044b4e042894d5,/modules/registry/registry-core/src/main/java/...,522,556,https://github.com/apache/airavata/blob/391843...,1.0


    In the next step we are going to download each of these datasets(smells) refrence code. Refrence code has been determined in the mentioned articles dataset in the link feature. In the following we have written a function that takes a url as input and finds the code and saves it into a file.

In [13]:
import requests
import lxml
from bs4 import BeautifulSoup

def getCodes(url, dframe, pathToSave, counter):
    global errorLinks
    headers = {
      'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36 QIHU 360SE'
    }
    req = requests.get(url, headers = headers)
    soup = BeautifulSoup(req.content,'lxml')
    code_id = list(dframe['id'])[counter]

    startLine = list(dframe["start_line"])[counter]
    EndLine = list(dframe["end_line"])[counter] # determined in the dataset
    
    file = open(f"{pathToSave}\\row-{counter}-id{code_id}.java", "w")
    t = True    
    for index in range(startLine, EndLine+1):
        if t:
            t = False
            pass
        else:
            file.write("\n")
        try:
            line = soup.find('td', {'id':f'LC{index}'}).get_text()
            file.write(line)
        except AttributeError:
            errorLinks.append(url)
            file.close()
            return
        except:
            errorLinks.append([url, "Other Errors"])
            file.close()
            return
    file.close()

In [18]:
# Defines with CSV
errorLinks = list()

pathToSaveCodeBlob = "D:\\Articles\\Code Smells\\CodeSmells\\Smell Project\\DatasetsCrawledCode\\Blob's code"
pathToSaveCodeDataClass = "D:\\Articles\\Code Smells\\CodeSmells\\Smell Project\\DatasetsCrawledCode\\Dataclass's code"
pathToSaveCodeFeatureEnvy = "D:\\Articles\\Code Smells\\CodeSmells\\Smell Project\\DatasetsCrawledCode\\FetureEnvy's code"
pathToSaveCodeLongMethod = "D:\\Articles\\Code Smells\\CodeSmells\\Smell Project\\DatasetsCrawledCode\\Longmethod's code"

# Blob Smell # # # # # # # # # # # # DONE # # # # # # # # # # # #
# Crawling Feature Envy's Code from the mentioned link in the dataset 
# cnt = 629
# for linkindex in range(629, len(dfBlob['link'])):
#     getCodes(list(dfBlob['link'])[linkindex], dfBlob, pathToSaveCodeBlob, cnt)
#     cnt += 1
#     if cnt == 53:
#         break
# Blob Smell # # # # # # # # # # DONE # # # # # # # # # # # # # # 










# # # # # # # # # # # # # # # Not DONE # # # # # # # # # # # # # # # 
# Crawling Feature Envy's Code from the mentioned link in the dataset 
# cnt = 629
# for linkindex in range(629, len(dfBlob['link'])):
#     getCodes(codeLink, dfDataClass, pathToSaveCodeDataClass)
#     getCodes(list(dfBlob['link'])[linkindex], dfBlob, pathToSaveCodeBlob, cnt)
#     cnt += 1
#     if cnt == 53:
#         break
# # # # # # # # # # # # # # # Not DONE # # # # # # # # # # # # # # # 

# getCodes(codeLink, dfLongMethod, pathToSaveCodeLongMethod)
# getCodes(codeLink, dfFeatureEnvy, pathToSaveCodeFeatureEnvy)

## Model prepration
    Now we're going to use these codes as our training set records and create a model to learn this smell. First we're working on blob smell.

In [ ]:
## Define Train and Test sets


## Use traing set to train your model


## Use Test set To check the accuracy